# Variable Preprocessing

In [ ]:
# install required libraries
!pip install xarray netCDF4 h5netcdf numpy

In [2]:
import xarray as xr
import numpy as np

### Load ERA5-Land and ERA5 data

In [3]:
era5land = xr.open_dataset('era5-land.nc') # open ERA5-Land NetCDF file
print(era5land) # view structure
print("\n")

era5 = xr.open_dataset("era5.nc") # open ERA5 NetCEF file
print(era5)

<xarray.Dataset> Size: 180MB
Dimensions:     (valid_time: 744, latitude: 96, longitude: 105)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 6kB 2019-08-01 ... 2019-08-31T23:...
    expver      (valid_time) <U4 12kB ...
  * latitude    (latitude) float64 768B 42.0 41.9 41.8 41.7 ... 32.7 32.6 32.5
  * longitude   (longitude) float64 840B -124.5 -124.4 -124.3 ... -114.2 -114.1
    number      int64 8B ...
Data variables:
    d2m         (valid_time, latitude, longitude) float32 30MB ...
    t2m         (valid_time, latitude, longitude) float32 30MB ...
    ssrd        (valid_time, latitude, longitude) float32 30MB ...
    u10         (valid_time, latitude, longitude) float32 30MB ...
    v10         (valid_time, latitude, longitude) float32 30MB ...
    sp          (valid_time, latitude, longitude) float32 30MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conv

### converting varaible units

In [4]:
# convert t2m from K to Celsius -- used as "Tair" input
t2m_c = era5land["t2m"] - 273.15
t2m_c.attrs["units"] = "C"

# convert d2m from K to Celsius -- used in "relhum" calculation
d2m_c = era5land["d2m"] - 273.15
d2m_c.attrs["units"] = "C"

# convert ssrd from J/m2 to W/m2 -- used as "solar" input
era5land["ssrd"].valid_time.diff("time") # check that time resolution is hourly
ssrd_W = era5land["ssrd"] / 3600         # dividy by 3600 seconds since hourly accumulated
ssrd_W.attrs["units"] = "W m**-2"

# convert sp from Pa to hPa -- used as "pres" input
sp_hpa = era5land["sp"] / 100
sp_hpa.attrs["units"] = "hPa"

### converting dimensions

In [ ]:
# convert time to local standard time (refer to Spangler's paper)

# convert longitude from 0–>360 to -180–>180

### derived variables

In [10]:
# cza -- Cosine solar zenith angle (0-1); use calc_cza_int() or calc_solar_parameters()$cza if cza is not known
# reference "heatmetrics" R package for calculation

# fdir -- Fraction of surface solar radiation that is direct (0-1)
# reference "heatmetrics" R package for calculation (or maybe just ERA5 fdir/ssdr)??
# remember that fdir is from ERA5; need to interpolate from 0.25 to 0.1 degree resolution

# calculate relative humidity (%) -- used as "relhum" input
vapor_pres = 610.94*np.exp(17.625*d2m_c / (243.04+d2m_c))
sat_vapor_pres = 610.94*np.exp(17.625*t2m_c / (243.04+t2m_c))
rh = 100*(vapor_pres/sat_vapor_pres)
rh.attrs["units"] = "%"

# speed -- Wind speed (m/s)
# reference Spangler's paper for calculation
u = era5land["u10"]
v = era5land["v10"]
ws = np.sqrt(u**2 + v**2)
ws.attrs["units"] = "m s**-1"

### obtain and derive "urban" variable

In [ ]:
# from National Land Cover Database

### build a working dataset containing wbgt input variables
xr.Dataset creates a dataset resembling an in-memory representation of a NetCDF file. Consists of variables, coordinates, and attributes, which together form a self-describing dataset.

In [ ]:
ds_work = xr.Dataset(
    {
        "Tair": t2m_c,
        "d2m": d2m_c,
        "solar": ssrd_W,
        "pres": sp_hpa,
        ... # ADD OTHER WBGT INPUT VARIABLES
    },
    coords=ds.coords,
)

print(ds_work)